In [400]:
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error
import pandas as pd
import numpy as np

pd.set_option('display.width', 500)

In [401]:
holidayEvents = pd.read_csv('/content/drive/MyDrive/Lumix/StoreSales-Kaggle/holidays_events.csv', index_col='date', parse_dates=['date']).drop(['description'], axis=1)
oil = pd.read_csv('/content/drive/MyDrive/Lumix/StoreSales-Kaggle/oil.csv', index_col='date', parse_dates=['date']).rename({'dcoilwtico': 'oil_price'}, axis=1)
stores = pd.read_csv('/content/drive/MyDrive/Lumix/StoreSales-Kaggle/stores.csv', index_col='store_nbr')
transactions = pd.read_csv('/content/drive/MyDrive/Lumix/StoreSales-Kaggle/transactions.csv', index_col='date', parse_dates=['date'])
test = pd.read_csv('/content/drive/MyDrive/Lumix/StoreSales-Kaggle/test.csv', index_col='date', parse_dates=['date']).drop(['id'], axis=1)
train = pd.read_csv('/content/drive/MyDrive/Lumix/StoreSales-Kaggle/train.csv', index_col='date', parse_dates=['date']).drop(['id'], axis=1)
sampleSubmission = pd.read_csv('/content/drive/MyDrive/Lumix/StoreSales-Kaggle/sample_submission.csv', index_col='id')

# Tratamento de dados de teste

In [402]:
train

,store_nbr,family,sales,onpromotion
date,,,,
2013-01-01,1,AUTOMOTIVE,0.000,0
2013-01-01,1,BABY CARE,0.000,0
2013-01-01,1,BEAUTY,0.000,0
2013-01-01,1,BEVERAGES,0.000,0
2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...
2017-08-15,9,POULTRY,438.133,0
2017-08-15,9,PREPARED FOODS,154.553,1
2017-08-15,9,PRODUCE,2419.729,148


In [403]:
transactionsReset = transactions.reset_index();
trainReset = train.reset_index();
oilReset = oil.reset_index();
holidaysReset = holidayEvents.reset_index();
storesReset = stores.reset_index();

data = trainReset.copy()

In [404]:
# MERGE TRANSACTIONS
data = data.merge(
    transactionsReset[['date', 'store_nbr', 'transactions']],
    on=['date', 'store_nbr'],
    how='left',
)
data.loc[data['transactions'].isna(), 'transactions'] = 0

In [405]:
# MERGE OIL PRICE
data = data.merge(
    oilReset[['date', 'oil_price']],
    on=['date'],
    how='left'
)
data.loc[data.index == data.index.min() ,'oil_price'] = 93.14
data['oil_price'] = data['oil_price'].interpolate(method='linear')

In [406]:
# MERGE STORES
storesReset.rename({
    'cluster': 'store_cluster',
    'type': 'store_type',
    'city': 'store_city',
    'state': 'store_state'
}, axis=1, inplace=True)
data = data.merge(
    storesReset[['store_nbr', 'store_cluster', 'store_type', 'store_city', 'store_state']],
    on=['store_nbr'],
    how='left'
)

In [407]:
# MERGE HOLIDAYS
CITY_COL = 'store_city'
STATE_COL = 'store_state'

nationalHolidays = holidaysReset[holidaysReset['locale'] == 'National'].copy().rename(columns={
    'type': 'type_nat',
    'transferred': 'transferred_nat'
})
data = pd.merge(
    data,
    nationalHolidays[['date', 'type_nat', 'transferred_nat']],
    on=['date'],
    how='left'
)
regionalHolidays = holidaysReset[holidaysReset['locale'] == 'Regional'].copy().rename(columns={
    'locale_name': STATE_COL,
    'type': 'type_reg',
    'transferred': 'transferred_reg'
})
data = pd.merge(
    data,
    regionalHolidays[['date', STATE_COL, 'type_reg', 'transferred_reg']],
    on=['date', STATE_COL],
    how='left'
)
localHolidays = holidaysReset[holidaysReset['locale'] == 'Local'].copy().rename(columns={
    'locale_name': CITY_COL,
    'type': 'type_loc',
    'transferred': 'transferred_loc'
})
data = pd.merge(
    data,
    localHolidays[['date', CITY_COL, 'type_loc', 'transferred_loc']],
    on=['date', CITY_COL],
    how='left'
)

data['holiday_type'] = data['type_nat'].fillna(data['type_reg']).fillna(data['type_loc'])
data['holiday_transferred'] = data['transferred_nat'].fillna(data['transferred_reg']).fillna(data['transferred_loc'])

data['day_type'] = data['holiday_type']
data['day_type'] = data['day_type'].replace(['Additional', 'Transfer', 'Bridge'], 'Holiday')

data.loc[(data['day_type'] == 'Holiday') & (data['holiday_transferred'] == True), 'day_type'] = 'Work Day'

is_null = data['day_type'].isna()
is_weekday = data['date'].dt.weekday < 5
data.loc[is_null & is_weekday, 'day_type'] = 'Work Day'
data.loc[is_null & ~is_weekday, 'day_type'] = 'Weekend'

data.drop(['type_loc', 'type_reg', 'type_nat', 'transferred_loc', 'transferred_reg', 'transferred_nat', 'holiday_type', 'holiday_transferred'], axis=1, inplace=True)

In [408]:
# CONVERT CATEGORICAL COLUMNS TO NUMBERS
categorical_cols = ['family', 'store_type', 'store_city', 'store_state', 'day_type']
encoders = {}
for col in categorical_cols:
    encoder = LabelEncoder()
    encoders[col] = encoder
    data[col] = encoder.fit_transform(data[col])

In [409]:
# REMOVE DUPES ( ADDING THERE SOMEWHERE )
data = data.drop_duplicates(subset=['date', 'store_nbr', 'family'], keep='first')

In [410]:
# RESTORE INDEX
data.set_index('date', inplace=True)

# Dados auxiliares

**DATA DE PREVISÃO:** 16/08/2017 a 31/08/2017

In [411]:
# OIL IN TEST RANGE
oilTestRange = oil.loc['2017-08-16':]

full_range = pd.date_range(start=oilTestRange.index.min(), end=oilTestRange.index.max())
oilTestRange = oilTestRange.reindex(full_range)

oilTestRange = oilTestRange.interpolate(method='linear')

oilTestRange.index.name = 'date'

In [412]:
# HOLIDAYS IN TEST RANGE
holidaysTestRange = holidayEvents.loc['2017-08-16':'2017-09-01']
holidaysTestRange

,type,locale,locale_name,transferred
date,,,,
2017-08-24,Holiday,Local,Ambato,False


In [413]:
# TRANSACTIONS PREDICTION
horizon = 16
transactions_forecasts = []

transactions = data.reset_index()
transactions.drop_duplicates(subset=['date', 'store_nbr'], inplace=True)
transactions.drop(columns=['onpromotion', 'sales', 'family'], inplace=True)
future_dates = pd.date_range(transactions['date'].max() + pd.Timedelta(days=1), periods=horizon)
transactions.set_index('date', inplace=True)

store_family_map = data[['store_nbr', 'family']].drop_duplicates()

for store in sorted(transactions['store_nbr'].unique()):
  store_data = transactions[transactions['store_nbr'] == store]
  store_data = store_data.asfreq('D').fillna(0)
  y = store_data['transactions']

  fourier = CalendarFourier(freq="W", order=10)
  dp = DeterministicProcess(
      index=store_data.index,
      order=8,
      seasonal=True,
      additional_terms=[fourier],
  )
  x = dp.in_sample();
  linear_model = LinearRegression()
  linear_model.fit(x, y)
  y_pred_lr = linear_model.predict(x)

  x = store_data.drop(columns=['transactions'])
  tree_model = XGBRegressor()
  tree_model.fit(x, y - y_pred_lr)

  x_future = dp.out_of_sample(steps=horizon)
  y_pred_lr_future = linear_model.predict(x_future)
  last_row = x.iloc[[-1]].copy()
  x_future_tree = pd.concat([last_row]*horizon, ignore_index=True)
  y_pred_xg_future = tree_model.predict(x_future_tree)
  y_pred_future = y_pred_lr_future + y_pred_xg_future

  store_families = store_family_map[store_family_map['store_nbr'] == store]['family']
  for fam in store_families:
      store_result = pd.DataFrame({
          'date': future_dates,
          'store_nbr': store,
          'family': fam,
          'transactions': y_pred_future
      })
      transactions_forecasts.append(store_result)

forecastTransactions = pd.concat(transactions_forecasts, ignore_index=True)
forecastTransactions['transactions'] = forecastTransactions['transactions'].round().abs()

# Tratamento dos dados de previsão

In [414]:
testReset = test.reset_index()
oilTestRangeReset = oilTestRange.reset_index()
holidaysTestRangeReset = holidaysTestRange.reset_index()
forecastTransactionsReset = forecastTransactions.reset_index()

testData = testReset.copy()

In [415]:
# MERGE OIL PRICE
testData = testData.merge(
    oilTestRangeReset[['date', 'oil_price']],
    on=['date'],
    how='left'
)

In [416]:
# MERGE STORES
storesReset.rename(columns={
    'cluster': 'store_cluster',
    'type': 'store_type',
    'city': 'store_city',
    'state': 'store_state'
}, inplace=True)
testData = testData.merge(
    storesReset[['store_nbr', 'store_cluster', 'store_type', 'store_city', 'store_state']],
    on=['store_nbr'],
    how='left'
)

In [417]:
# MERGE HOLIDAYS
CITY_COL = 'store_city'
STATE_COL = 'store_state'

nationalHolidays = holidaysReset[holidaysReset['locale'] == 'National'].copy().rename(columns={
    'type': 'type_nat',
    'transferred': 'transferred_nat'
})
testData = pd.merge(
    testData,
    nationalHolidays[['date', 'type_nat', 'transferred_nat']],
    on=['date'],
    how='left'
)
regionalHolidays = holidaysReset[holidaysReset['locale'] == 'Regional'].copy().rename(columns={
    'locale_name': STATE_COL,
    'type': 'type_reg',
    'transferred': 'transferred_reg'
})
testData = pd.merge(
    testData,
    regionalHolidays[['date', STATE_COL, 'type_reg', 'transferred_reg']],
    on=['date', STATE_COL],
    how='left'
)
localHolidays = holidaysReset[holidaysReset['locale'] == 'Local'].copy().rename(columns={
    'locale_name': CITY_COL,
    'type': 'type_loc',
    'transferred': 'transferred_loc'
})
testData = pd.merge(
    testData,
    localHolidays[['date', CITY_COL, 'type_loc', 'transferred_loc']],
    on=['date', CITY_COL],
    how='left'
)

testData['holiday_type'] = testData['type_nat'].fillna(testData['type_reg']).fillna(testData['type_loc'])
testData['holiday_transferred'] = testData['transferred_nat'].fillna(testData['transferred_reg']).fillna(testData['transferred_loc'])

testData['day_type'] = testData['holiday_type']
testData['day_type'] = testData['day_type'].replace(['Additional', 'Transfer', 'Bridge'], 'Holiday')

testData.loc[(testData['day_type'] == 'Holiday') & (testData['holiday_transferred'] == True), 'day_type'] = 'Work Day'

is_null = testData['day_type'].isna()
is_weekday = testData['date'].dt.weekday < 5
testData.loc[is_null & is_weekday, 'day_type'] = 'Work Day'
testData.loc[is_null & ~is_weekday, 'day_type'] = 'Weekend'

testData.drop(['type_loc', 'type_reg', 'type_nat', 'transferred_loc', 'transferred_reg', 'transferred_nat', 'holiday_type', 'holiday_transferred'], axis=1, inplace=True)

/tmp/ipython-input-417-2079727813.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  testData['holiday_type'] = testData['type_nat'].fillna(testData['type_reg']).fillna(testData['type_loc'])
/tmp/ipython-input-417-2079727813.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  testData['holiday_transferred'] = testData['transferred_nat'].fillna(testData['transferred_reg']).fillna(testData['transferred_loc'])


In [418]:
# CONVERT CATEGORICAL COLUMNS TO NUMBERS
for col in categorical_cols:
    encoder = encoders[col]
    testData[col] = encoder.transform(testData[col])

In [419]:
# MERGE TRANSACTIONS
testData = testData.merge(
    forecastTransactionsReset[['date', 'store_nbr', 'family', 'transactions']],
    on=['date', 'store_nbr', 'family'],
    how='left'
)

In [420]:
# RESTORE INDEX
testData.set_index('date', inplace=True)

In [421]:
train.shape

(3000888, 4)

In [422]:
data.shape

(3000888, 11)

In [423]:
test.shape

(28512, 3)

In [424]:
testData.shape

(28512, 10)

# Predição

In [434]:
horizon = 16
linearModelOrder = 8
sales_forecasts = []

store_family_list = sorted(data[['store_nbr', 'family']].drop_duplicates().values.tolist())

for store, family in store_family_list:
    store_data = data[(data['store_nbr'] == store) & (data['family'] == family)].copy()

    store_data.index = pd.to_datetime(store_data.index)
    store_data = store_data.asfreq('D').fillna(0)

    y = store_data['sales']

    fourier = CalendarFourier(freq="W", order=10)
    dp = DeterministicProcess(
        index=store_data.index,
        order=linearModelOrder,
        seasonal=True,
        additional_terms=[fourier],
    )
    x_lin = dp.in_sample()
    linear_model = LinearRegression()
    linear_model.fit(x_lin, y)
    y_pred_lr = linear_model.predict(x_lin)

    x_tree = store_data.drop(columns=['sales'])
    tree_model = XGBRegressor()
    tree_model.fit(x_tree, y - y_pred_lr)

    y_pred_tree = tree_model.predict(x_tree)

    future_idx = pd.date_range(start=store_data.index[-1] + pd.Timedelta(days=1), periods=horizon)
    x_future_lin = dp.out_of_sample(steps=horizon)
    y_future_lr = linear_model.predict(x_future_lin)

    x_future_tree = pd.DataFrame([x_tree.iloc[-1].values] * horizon, columns=x_tree.columns, index=future_idx)
    y_future_tree = tree_model.predict(x_future_tree)

    y_total = pd.Series(y_future_lr + y_future_tree, index=future_idx)

    forecast = pd.DataFrame({
        'date': y_total.index,
        'store_nbr': store,
        'family': family,
        'sales': y_total
    })
    sales_forecasts.append(forecast)

forecastResult = pd.concat(sales_forecasts, ignore_index=True)

In [437]:
forecastBackup = forecastResult.copy()
forecastBackup

,date,store_nbr,family,sales
0,2017-08-16,1,0,3.317095
1,2017-08-17,1,0,3.346605
2,2017-08-18,1,0,3.376463
3,2017-08-19,1,0,3.406671
4,2017-08-20,1,0,3.437231
...,...,...,...,...
28507,2017-08-27,54,32,2.834764
28508,2017-08-28,54,32,2.836658
28509,2017-08-29,54,32,2.838438
28510,2017-08-30,54,32,2.840106


In [439]:
forecastResult['sales'] = forecastResult['sales'].round().abs()
forecastResult = forecastResult.sort_values(
    by=['date', 'store_nbr', 'family'],
    key=lambda col: (
        col if col.name != 'date' else forecastResult['store_nbr'] * 10000 + pd.to_datetime(col).factorize()[0]
    )
).reset_index(drop=True)

In [441]:
forecastBackup = forecastResult.copy()

In [442]:
forecastResult = forecastResult.set_index('date')

In [443]:
forecastResult

,store_nbr,family,sales
date,,,
2017-08-16,1,0,3.0
2017-08-16,1,1,0.0
2017-08-16,1,2,5.0
2017-08-16,1,3,1964.0
2017-08-16,1,4,0.0
...,...,...,...
2017-08-31,54,28,58.0
2017-08-31,54,29,107.0
2017-08-31,54,30,937.0


In [445]:
forecastSend = forecastResult['sales']
forecastSend

,sales
date,
2017-08-16,3.0
2017-08-16,0.0
2017-08-16,5.0
2017-08-16,1964.0
2017-08-16,0.0
...,...
2017-08-31,58.0
2017-08-31,107.0
2017-08-31,937.0


In [447]:
forecastSend = forecastSend.reset_index(drop=True)
forecastSend.index = range(3000888, 3000888 + len(forecastSend))
forecastSend

,sales
3000888,3.0
3000889,0.0
3000890,5.0
3000891,1964.0
3000892,0.0
...,...
3029395,58.0
3029396,107.0
3029397,937.0
3029398,1.0


In [449]:
forecastSend.to_csv('data.csv')
!cp data.csv "/content/drive/My Drive/"

In [446]:
sampleSubmission

,sales
id,
3000888,0.0
3000889,0.0
3000890,0.0
3000891,0.0
3000892,0.0
...,...
3029395,0.0
3029396,0.0
3029397,0.0
